In [452]:
import pandas as pd

In [453]:
file_path = r"C:\Windows\System32\battery-report.html"
history = pd.read_html(file_path)[2]
history = history[history['STATE'] != 'Suspended'].drop(["CAPACITY REMAINING.1", "STATE"], axis = 1)
map = {
    "AC": 1,
    "Battery": 0
}
history["SOURCE"] = history["SOURCE"].map(map).astype(float)
history["CAPACITY REMAINING"] = history["CAPACITY REMAINING"].str[:-2].astype(float)
dt_full = pd.to_datetime(history['START TIME'], errors = "coerce")
history["DATE"] = dt_full.dt.day_name().ffill()
history['TIME'] = pd.to_datetime(history['START TIME'], format = "mixed")
map2 = {
    "Sunday": 1,
    "Monday": 2,
    "Tuesday": 3,
    "Wednesday": 4,
    "Thursday": 5,
    "Friday": 6,
    "Saturday": 7
}
history = history.drop("START TIME", axis = 1)
history["DATE"] = history["DATE"].map(map2).astype(float)
history["TIME"] = (history["TIME"].dt.second + history["TIME"].dt.minute * 60 + history["TIME"].dt.hour * 3600) * 0.0002
history

,SOURCE,CAPACITY REMAINING,DATE,TIME
0,1.0,69.0,7.0,6.6764
1,0.0,99.0,7.0,7.4066
2,1.0,99.0,7.0,7.4172
3,1.0,100.0,7.0,8.0010
4,1.0,100.0,7.0,8.0604
...,...,...,...,...
204,0.0,55.0,5.0,17.1482
205,0.0,55.0,5.0,17.1724
207,0.0,53.0,5.0,6.2684
208,0.0,53.0,5.0,6.2754


In [454]:
import torch
class chargenet(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.net = torch.nn.Sequential(
            torch.nn.Linear(3, 16),
            torch.nn.ReLU(),
            torch.nn.Linear(16, 8),
            torch.nn.ReLU(),
            torch.nn.Linear(8, 2)
        )
    def forward(self, x):
        return self.net(x)
model = chargenet()
lossfn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0005)

In [455]:
X = torch.from_numpy(history.drop("SOURCE", axis = 1).astype("float32").values)
Y = torch.from_numpy(history["SOURCE"].astype("int64").values)

In [456]:
for i in range(10000):
    model.train()
    optimizer.zero_grad()
    pred = model(X)
    loss = lossfn(pred, Y)
    loss.backward()
    optimizer.step()
    print(f"epoch #{i} has loss {loss.item()}")

epoch #0 has loss 1.4519838094711304
epoch #1 has loss 1.4099922180175781
epoch #2 has loss 1.3682805299758911
epoch #3 has loss 1.3268717527389526
epoch #4 has loss 1.2857898473739624
epoch #5 has loss 1.2450555562973022
epoch #6 has loss 1.2046993970870972
epoch #7 has loss 1.1647725105285645
epoch #8 has loss 1.125322699546814
epoch #9 has loss 1.0863689184188843
epoch #10 has loss 1.047977089881897
epoch #11 has loss 1.0102074146270752
epoch #12 has loss 0.9731285572052002
epoch #13 has loss 0.9368194341659546
epoch #14 has loss 0.9013684391975403
epoch #15 has loss 0.8668834567070007
epoch #16 has loss 0.8334948420524597
epoch #17 has loss 0.8013136386871338
epoch #18 has loss 0.7704782485961914
epoch #19 has loss 0.7412592768669128
epoch #20 has loss 0.713711142539978
epoch #21 has loss 0.6880084872245789
epoch #22 has loss 0.6644208431243896
epoch #23 has loss 0.6437283158302307
epoch #24 has loss 0.6254871487617493
epoch #25 has loss 0.6093955039978027
epoch #26 has loss 0.5957

In [457]:
model.eval()
correct = total = 0
with torch.no_grad():
    logits = model(X)
    preds = logits.argmax(dim=1)
    correct += (preds == Y).sum().item()
    total += Y.size(0)
print(f"Test Accuracy: {correct/total*100:.2f}%")

Test Accuracy: 84.34%
